# Guia para a aula experimental
Neste guia trataremos do uso de um sensor virtual, que utiliza fotografias para o cálculo de pessoas em uma localidade. Nesse guia vocês poderão testar diversos modelos de Deep Learning baseados na rede Yolo em diversas fotografias diferentes.

Para rodar esse Guia, você pode executar as células sequencialmente (exceto as que contêm listas de seleção).

Esse roteiro utiliza material do trabalho publicado pelo Google, que pode ser encontrado em: https://docs.ultralytics.com/modes/predict/

Licensed under the Apache License, Version 2.0

In [ ]:
%%capture
!pip install ultralytics

In [ ]:
import numpy as np
from ultralytics import YOLO
import scipy.misc
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
def load_image_into_numpy_array(path):
  image = None
  if(path.startswith('http')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))

  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)


def plot_person_boxes(image, results):
    # Criar uma figura e eixo para exibir a imagem (aumentando o tamanho da figura)
    fig, ax = plt.subplots(1, figsize=(12, 8))

    # Exibir a imagem
    ax.imshow(image)

    # Obter o mapeamento dos nomes das classes
    class_names = results.names  # Dicionário que mapeia índices de classe para nomes

    # Obter as caixas dos resultados
    boxes = results.boxes  # Objeto 'Boxes'

    # Verificar se há detecções
    if boxes is None or len(boxes) == 0:
        print("Nenhuma detecção para exibir.")
        plt.show()
        return

    # Converter caixas e índices de classe para arrays numpy
    xyxy = boxes.xyxy.cpu().numpy()  # Coordenadas das caixas [x1, y1, x2, y2]
    cls_indices = boxes.cls.cpu().numpy().astype(int)  # Índices das classes

    # Iterar sobre cada detecção
    for i in range(len(boxes)):
        # Obter o índice e nome da classe
        cls_idx = cls_indices[i]
        cls_name = class_names.get(cls_idx, 'Desconhecido')

        # Verificar se a classe detectada é 'person'
        if cls_name == 'person':
            # Obter as coordenadas da caixa delimitadora
            x1, y1, x2, y2 = xyxy[i]

            # Calcular a largura e altura da caixa
            width = x2 - x1
            height = y2 - y1

            # Criar um retângulo com as coordenadas da caixa
            rect = patches.Rectangle(
                (x1, y1), width, height,
                linewidth=2, edgecolor='r', facecolor='none'
            )

            # Adicionar o retângulo ao eixo
            ax.add_patch(rect)

    # Mostrar o gráfico com as caixas delimitadoras
    plt.show()


ALL_MODELS = {
'Yolo 11 - Nano' : 'yolo11n.pt',
'Yolo 11 - Small' : 'yolo11s.pt',
'Yolo 11 - Medium' : 'yolo11m.pt',
'Yolo 11 - Large' : 'yolo11l.pt',
'Yolo 11 - Extra' : 'yolo11x.pt'
}

IMAGES_FOR_TEST = {
  'Toronto' : 'https://wp222203.wpdns.ca/wp-content/uploads/2019/08/Kristen-Wells-Walking-tour-009_web-1024x683.jpg',
  'dreamstime.com' : 'https://thumbs.dreamstime.com/z/nyc-people-waiting-outside-restaurant-table-20995159.jpg',
  'BostonGlobe' : 'https://bostonglobe-prod.cdn.arcpublishing.com/resizer/kOd-PMQUsCOkoEqK861NUX1XDQ4=/1440x0/arc-anglerfish-arc2-prod-bostonglobe.s3.amazonaws.com/public/2P7FQXFU3EI6HLDEZS3P4FVF7I.jpg',
  'Multidão 1' : 'https://i.ytimg.com/vi/NzAt-73Q8U4/maxresdefault.jpg',
  'Multidão 2' : 'https://c8.alamy.com/comp/E53RWC/a-small-crowd-watching-a-band-performing-at-the-sound-of-music-festival-E53RWC.jpg',
}

In [ ]:
#Agora vamos selecionar o modelo a ser usado
#@title Seleção do modelo { display-mode: "form", run: "auto" }
model_display_name = 'Yolo 11 - Extra' # @param ['Yolo 11 - Nano','Yolo 11 - Small','Yolo 11 - Medium','Yolo 11 - Large', 'Yolo 11 - Extra']
model_handle = ALL_MODELS[model_display_name]

print('Modelo Selecionado:'+ model_display_name)

In [ ]:
# Baixando e carregando o modelo selecionado
model = YOLO(model_handle)

In [ ]:
#@title Seleção da Imagem { display-mode: "form", run: "auto" }
selected_image = 'Multidão 2' # @param ['Toronto', 'dreamstime.com', 'BostonGlobe', 'Multidão 1', 'Multidão 2']
flip_image_horizontally = False #@param {type:"boolean"}
convert_image_to_grayscale = False #@param {type:"boolean"}


In [ ]:
image_path = IMAGES_FOR_TEST[selected_image]
print('Baixando: '+ selected_image + ' , em: '+ image_path)
image_np = load_image_into_numpy_array(image_path)[0]


# Flip horizontally
if(flip_image_horizontally):
  image_np = np.fliplr(image_np[0]).copy()

# Convert image to grayscale
if(convert_image_to_grayscale):
  image_np = np.tile(
    np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

plt.figure(figsize=(12,16))
plt.imshow(image_np)
plt.show()

In [ ]:
# Rodando a Inferência do modelo escolhido na Imagem
results = model(image_np,conf=0.5)

#Exibindo o resultado
plot_person_boxes(image_np, results[0])